# Replicate the 2019-04-08 imagenette results

| Size (px)|Epochs|Accuracy|URL|Params|Notes|
|----------|------|--------|---|------|-----|
|128 |5|84.6|[link](https://github.com/fastai/imagenette/tree/dfdc53e4e0ffd620f42d0532981ddde7d6554059)|--epochs 5 --bs 64 --lr 3e-3 --mixup 0|4 GPUs|

Can anyone help explain why I'm getting nearly 10% lower accuracy?

Here's what I've done;
- created a new conda env and set-up fastai following https://github.com/fastai/fastai/blob/master/README.md#installation
- copied https://github.com/fastai/fastai/blob/master/examples/train_imagenette.py into this notebook
- removed `@call_parse` ad updated `main` params directly
    - I'll try to find time to get `@call_parse` working in the notbook soon
- run on a single GPU Ubuntu laptop

In [1]:
import fastai
fastai.__version__

'1.0.60'

In [2]:
from fastai.script import *
from fastai.vision import *
from fastai.callbacks import *
from fastai.distributed import *
from fastprogress import fastprogress
from torchvision.models import *
from fastai.vision.models.xresnet import *
from fastai.vision.models.xresnet2 import *
from fastai.vision.models.presnet import *

In [3]:
torch.backends.cudnn.benchmark = True
fastprogress.MAX_COLS = 80

In [4]:
def get_data(size, woof, bs, workers=None):
    if   size<=128: path = URLs.IMAGEWOOF_160 if woof else URLs.IMAGENETTE_160
    elif size<=224: path = URLs.IMAGEWOOF_320 if woof else URLs.IMAGENETTE_320
    else          : path = URLs.IMAGEWOOF     if woof else URLs.IMAGENETTE
    path = untar_data(path)

    n_gpus = num_distrib() or 1
    if workers is None: workers = min(8, num_cpus()//n_gpus)

    return (ImageList.from_folder(path).split_by_folder(valid='val')
            .label_from_folder().transform(([flip_lr(p=0.5)], []), size=size)
            .databunch(bs=bs, num_workers=workers)
            .presize(size, scale=(0.35,1))
            .normalize(imagenet_stats))

In [5]:
# --epochs 5 --bs 64 --lr 3e-3 --mixup 0
def main(
        gpu:Param("GPU to run on", str)=None,
        woof: Param("Use imagewoof (otherwise imagenette)", int)=0,
        lr: Param("Learning rate", float)=3e-3,
        size: Param("Size (px: 128,192,224)", int)=128,
        alpha: Param("Alpha", float)=0.99,
        mom: Param("Momentum", float)=0.9,
        eps: Param("epsilon", float)=1e-6,
        epochs: Param("Number of epochs", int)=5,
        bs: Param("Batch size", int)=64,
        mixup: Param("Mixup", float)=0.,
        opt: Param("Optimizer (adam,rms,sgd)", str)='adam',
        arch: Param("Architecture (xresnet34, xresnet50, presnet34, presnet50)", str)='xresnet50',
        dump: Param("Print model; don't train", int)=0,
        ):
    "Distributed training of Imagenette."

    gpu = setup_distrib(gpu)
    if gpu is None: bs *= torch.cuda.device_count()
    if   opt=='adam' : opt_func = partial(optim.Adam, betas=(mom,alpha), eps=eps)
    elif opt=='rms'  : opt_func = partial(optim.RMSprop, alpha=alpha, eps=eps)
    elif opt=='sgd'  : opt_func = partial(optim.SGD, momentum=mom)

    data = get_data(size, woof, bs)
    bs_rat = bs/256
    if gpu is not None: bs_rat *= num_distrib()
    if not gpu: print(f'lr: {lr}; eff_lr: {lr*bs_rat}; size: {size}; alpha: {alpha}; mom: {mom}; eps: {eps}')
    lr *= bs_rat

    m = globals()[arch]
    learn = (Learner(data, m(c_out=10), wd=1e-2, opt_func=opt_func,
             metrics=[accuracy,top_k_accuracy],
             bn_wd=False, true_wd=True,
             loss_func = LabelSmoothingCrossEntropy())
            )
    if dump: print(learn.model); exit()
    if mixup: learn = learn.mixup(alpha=mixup)
    learn = learn.to_fp16(dynamic=True)
    if gpu is None:       learn.to_parallel()
    elif num_distrib()>1: learn.to_distributed(gpu) # Requires `-m fastai.launch`

    learn.fit_one_cycle(epochs, lr, div_factor=10, pct_start=0.3)
    
main()

lr: 0.003; eff_lr: 0.00075; size: 128; alpha: 0.99; mom: 0.9; eps: 1e-06


epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,1.719871,2.395309,0.366115,0.843567,00:27
1,1.502559,1.635979,0.563312,0.918726,00:24
2,1.319078,1.429097,0.644586,0.948025,00:24
3,1.175530,1.212281,0.715924,0.966115,00:25
4,1.077183,1.132247,0.751338,0.975796,00:25
